In [15]:
from sympy.solvers import solve
from sympy import Symbol
from fractions import Fraction
import math

In [16]:
# Functions to perform weiner attack

# euclidean alogrithm to obtain continued fraction expansion of x/y
def partial_quotients(x, y):
    pq = []
    while x != 1:
        pq.append(x // y)
        a = y
        b = x % y
        x = a
        y = b
    return pq

# obtaining rational convergent
def rational(pq):
    i = len(pq) - 1
    num = pq[i]
    denom = 1
    while i > 0:
        i -= 1
        a = (pq[i] * num) + denom
        b = num
        num = a
        denom = b
    return (num, denom)

# getting 1st to last convergent of the continued fraction
def convergents(pq):
    c = []
    for i in range(1, len(pq)):
        c.append(rational(pq[0:i]))
    return c

# obtaining phi(n) from the approximated k/d value
def phiN(e, d, k):
    phiN = ((Fraction(e) * Fraction(d)) - 1)/Fraction(k)
    return phiN

In [17]:
def string_to_int(s):
    return int.from_bytes(s.encode(), byteorder='little')

def int_to_string(i):
    length = math.ceil(i.bit_length() / 8)
    return i.to_bytes(length, byteorder='little').decode()

In [18]:
plaintext = string_to_int("hello world")
print(plaintext)

121404708502361365413651816


In [19]:
# values of e and n that are susceptible to Weiner's attack\

# e = 17993
# n = 90581

e = 318540665379393469901456665807211509077755719995811520039095212139429238053864597311950397094944291616119321660193803737677538864969915331331528398734504661147661499115125056479426948683504604460936703005724827506058051215012025774714463561829608252938657297504427643593752676857551877096958959488289759878259498255905255543409142370769036479607835226542428818361327569095305960454592450213005148130508649794732855515489990191085723757628463901282599712670814223322126866814011761400443596552984309315434653984387419451894484613987942298157348306834118923950284809853541881602043240244910348705406353947587203832407
n = 338630205260455689413627911306068443537112802550361922213620660503310212139001530156458392949653034244789612680980241965923780722889133495349537107789761426092510299239678696031652780059016898519278860185536978111680123402473365833456785718098200501968322228116681190425490850863660038143310790555506293106653050174262471649179173093656763946257235681980586392230447218179278964626176124426615857733950102117938674282636936094069075258237416065546593509302494726576026227551920883962084579635168761189995794814926094510046419165007371450799003658587100556051088147493947712592469412133312536422828670173807709914587

In [20]:
ciphertext = pow(plaintext, e, n)

In [21]:
pq = partial_quotients(e, n)
c = convergents(pq)
x = Symbol('x')
for (k, d) in c:
    if k != 0:
        # y = p + q
        y = n - phiN(e, d, k) + 1
        # p, q will be roots of this eqn: (x-p)(x-q) = x^2 - (p+q)x + pq
        roots = solve(x ** 2 - y * x + n, x)
        if len(roots) == 2:
            p = roots[0]
            q = roots[1]
            if p * q == n:
                print(f'p = {p}\n')
                print(f'q = {q}\n')
                print("Found correct value of d!!!")
                print(f'd = {d}\n')
                obtained_plaintext = pow(ciphertext, d, n)
                #print(obtained_plaintext)
                obtained_plaintext = int_to_string(obtained_plaintext)
                print(f'Hacked plaintext: {obtained_plaintext}')
                break

p = 12001304129015480165432875074437607933493850611499879464845243350215176144760883615322622081442653872645865326992384034722586201972392183010813439352778246403016897976571514715418700569567613729681273931557848857971070286176848136118602099586101089743239644367344468295964691411425416652519752140536869089101

q = 28216117316929874067495888027767527011360661622486842768414059951572932145196930641365509243766454218518793508840136548374994021850853203018205749779390383366761851772055038753940967432004901699256177783249460134792699230632136386268348434203012426963129659057781488950062703849444443906614331812260961682887

Found correct value of d!!!
d = 724746542590011388513367385228693742222740657137483753552318433232068370338961145215199994578740789016238655979015224570943

Hacked plaintext: hello world
